In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import timedelta, date
from ast import literal_eval

'''This script produces a file which contains the book-arrive delta for each transaction.'''

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

# load purchase data
purchases_df = pd.read_csv('../../../data/cabot_data/clean_base/2018_clean.csv',
                           parse_dates=['ARRIVAL', 'DEPART', 'LOOK_DATE'])
purchases_df = (purchases_df[purchases_df['CANCEL_INDICATOR'] == 0] # look at bookings only
                .drop(['RESNO', 'CANCEL_INDICATOR'], axis=1)
                .groupby('group_id').first() # collapse groups
                .sort_values('LOOK_DATE'))

# add stay len col to df_data
purchases_df['stay_len'] = (purchases_df['DEPART'] - purchases_df['ARRIVAL']).dt.days
# anything greater than 4 is 4
purchases_df['stay_len'] = purchases_df['stay_len'].apply(lambda x: 4 if x > 4 else x)

# initialize some helper vars
df_grouped = purchases_df.groupby(by=['LOOK_DATE']).count()
periods_per_day = df_grouped['DEPART'].max()

look_start = purchases_df['LOOK_DATE'].min()
look_end = purchases_df['LOOK_DATE'].max()

ssn_start = purchases_df['ARRIVAL'].min()
ssn_end = purchases_df['ARRIVAL'].max()